<a href="https://colab.research.google.com/github/amanbs/career/blob/master/ECG_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Importing data into colab
